In [1]:
from time import sleep
import pyautogui
from os import path
from random import uniform, seed
import math

In [2]:
seed()

In [25]:
ANIM_SECONDS = 0.01
CUR_DIR = "C:\\Users\\Shevis\\src\\github.com\\shevisj\\splitbuilder"
BUILD_MODE = "move_speed.png"
OBJECTS_MENU = "objects_menu.png"
TOOLS_MENU = "tools_menu.png"
MOVEMENT_MENU = "movement_menu.png"
ROTATION_MENU = "rotation_menu.png"
BLOCKS_MENU = "blocks_menu.png"
RAMPS_MENU = "blocks_ramps_menu.png"
GLASS_RAMP = "glass_ramp.png"

In [4]:
X_POS = (216, 935)
Y_POS = (216, 1107)
Z_POS = (216, 1280)

# Start Game

In [5]:
def find_icon(icon_name, no_print=False):
    # Find the game icon
    image_path = path.join(CUR_DIR, 'images', icon_name)
    try:
        game_icon = pyautogui.locateOnScreen(image_path, confidence=0.9)
        if game_icon:
            if not no_print:
                print(f"{icon_name} found: {game_icon}")
            return pyautogui.center(game_icon)
        return None
    except pyautogui.ImageNotFoundException:
        if not no_print:
            print("Unable to locate image file")
        return None

def click_icon(icon_name, no_print=False):
    game_icon = find_icon(icon_name=icon_name, no_print=no_print)
    if game_icon:
        # Start the game
        pyautogui.moveTo(game_icon.x, game_icon.y, duration=ANIM_SECONDS)
        pyautogui.click(game_icon)
    elif not no_print:
        print("Unable to locate game icon", icon_name)

def click_spot(x, y, dur=ANIM_SECONDS):
    pyautogui.moveTo(x, y, duration=dur)
    pyautogui.click(x=x, y=y)

# Movement

In [10]:
def walk_wasd(dur=2):
    pyautogui.keyDown('w')
    sleep(dur)
    pyautogui.keyUp('w')

# Building

In [91]:
def is_active(img):
    icon_path = path.join(CUR_DIR, 'images', img)
    icon = pyautogui.locateOnScreen(icon_path, confidence=0.9)
    return icon is not None

def activate_build_mode():
    if not is_active(BUILD_MODE):
        pyautogui.press('`')

def open_build_menu():
    if not is_active(OBJECTS_MENU):
        pyautogui.press('m')

def enter_value(spot, val):
    click_spot(spot[0], spot[1])
    val_chars = list(str(val))
    for ch in val_chars:
        pyautogui.press(ch)
    pyautogui.press('enter')

def place_object(nav, loc, rot):
    click_icon(OBJECTS_MENU)
    for menu in nav:
        click_icon(menu, no_print=True)
    click_icon(TOOLS_MENU)
    click_icon(MOVEMENT_MENU)
    enter_value(X_POS, loc[0])
    enter_value(Y_POS, loc[1])
    enter_value(Z_POS, loc[2])
    click_icon(ROTATION_MENU)
    enter_value(X_POS, rot[0])
    enter_value(Y_POS, rot[1])
    enter_value(Z_POS, rot[2])
    click_spot(1122, 1122)

def straight_panal_extrapolator(start_loc, start_rot, n_panals):
    pos_x = start_loc[0]
    pos_y = start_loc[1]
    pos_z = start_loc[2]
    rot_x = start_rot[0]
    rot_y = start_rot[1]
    rot_z = start_rot[2]
    panal_width = 400
    sign = math.copysign(1, n_panals)
    panal_mag = abs(n_panals)
    def extrapolate():
        nonlocal pos_x, pos_z
        for _ in range(panal_mag - 1):
            pos_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y)) * panal_width * sign)
            pos_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y)) * panal_width * sign)
            yield (pos_x, pos_y, pos_z), (rot_x, rot_y, rot_z)
    return extrapolate

def place_surf_straight(start_loc, start_rot, n_panals):
    place_object(
        [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP],
        start_loc,
        start_rot
    )
    spe = straight_panal_extrapolator(start_loc=start_loc, start_rot=start_rot, n_panals=n_panals)
    for loc, rot in spe():
        place_object(
            [GLASS_RAMP],
            loc,
            rot
        )

def calc_sep_deg(radius):
    return math.degrees(math.atan(400.0 / (float(radius) + (800 * math.cos(math.radians(45))))))

# Radii is a list of 2-tuples with radius, count pairs
def flex_curv_extrapolator(start_loc, start_rot, radii, sign=1):
    pos_x = start_loc[0]
    pos_y = start_loc[1]
    pos_z = start_loc[2]
    rot_x = start_rot[0]
    rot_y = start_rot[1]
    rot_z = start_rot[2]
    n_panals = sum([r[1] for r in radii])
    def extrapolate():
        cur_pos_x = pos_x
        cur_pos_z = pos_z
        cur_rot_y = rot_y
        for cur_radius in radii:
            cur_center_x = cur_pos_x + round(math.sin(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0] * -1.0)
            cur_center_z = cur_pos_z + round(math.cos(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0] * -1.0)
            deg_offset = calc_sep_deg(cur_radius[0]) * sign
            for _ in range(cur_radius[1]):
                cur_rot_y = mod_rot(round(cur_rot_y + deg_offset))
                cur_pos_x = cur_center_x + round(math.sin(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0])
                cur_pos_z = cur_center_z + round(math.cos(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0])
                yield (cur_pos_x, pos_y, cur_pos_z), (rot_x, cur_rot_y, rot_z)
    return extrapolate

def place_flex_curv(start_loc, start_rot, radii, sign=1, ignore_first=False):
    if not ignore_first:
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP],
            start_loc,
            start_rot
        )
    fce = flex_curv_extrapolator(start_loc, start_rot, radii, sign)
    first = True
    for loc, rot in fce():
        print(loc, rot)
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP] if ignore_first and first else [GLASS_RAMP],
            loc,
            rot
        )
        first = False

def mod_rot(rot):
    currot = rot
    while currot < -180:
        currot += 360
    return ((rot + 180) % 360) - 180
        

def curv_extrapolator(start_loc, start_rot, radius, n_panals=5, h_delta=0, r_delta=0):
    pos_x = start_loc[0]
    pos_y = start_loc[1]
    pos_z = start_loc[2]
    rot_x = start_rot[0]
    rot_y = start_rot[1]
    rot_z = start_rot[2]
    abs_radius = abs(radius)
    sign = math.copysign(1, n_panals)
    abs_n_panals = abs(n_panals)
    center_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0)
    center_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0)
    def extrapolate():
        deg_offset = 0
        cur_pos_y = pos_y
        cur_abs_radius = abs_radius
        for _ in range(abs_n_panals - 1):
            
            cur_pos_y += h_delta
            cur_abs_radius += r_delta

            deg_offset += calc_sep_deg(cur_abs_radius) * sign
            new_x = center_x + round(math.sin(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius)
            new_z = center_z + round(math.cos(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius)
            yield (new_x, cur_pos_y, new_z), (rot_x, mod_rot(round(rot_y + deg_offset)), rot_z)
    return extrapolate

def place_curv(start_loc, start_rot, radius, n_panals=5, h_delta=0, r_delta=0, ignore_first=False):
    if not ignore_first:
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP],
            start_loc,
            start_rot
        )
    ce = curv_extrapolator(start_loc, start_rot, radius, n_panals, h_delta, r_delta)
    first = True
    for loc, rot in ce():
        print(loc, rot)
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP] if ignore_first and first else [GLASS_RAMP],
            loc,
            rot
        )
        first = False
    

# Runner

In [93]:
try:
    click_icon('game_icon.png')
    activate_build_mode()
    open_build_menu()
    place_flex_curv(
        (2646, 11890, -14235),
        (0, -45, -31),
        [
            (6000, 1),
            (5000, 2),
            (4000, 3),
            (3000, 4),
            (2000, 5),
            (3000, 4),
            (4000, 3),
            (5000, 2),
            (6000, 1),
        ]
    )
    #place_curv(
    #    (2646, 18890, -14235),
    #    (0, 100, -31),
    #    1500,
    #    28,
    #    -50,
    #    -50
    #)
except KeyboardInterrupt as e:
    print("Exiting")

game_icon.png found: Box(left=1321, top=2115, width=30, height=20)
objects_menu.png found: Box(left=121, top=372, width=144, height=24)
tools_menu.png found: Box(left=557, top=372, width=104, height=24)
movement_menu.png found: Box(left=401, top=573, width=142, height=20)
rotation_menu.png found: Box(left=628, top=573, width=125, height=20)
(2430, 11890, -14463) (0, -42, -31)
objects_menu.png found: Box(left=315, top=372, width=144, height=24)
tools_menu.png found: Box(left=557, top=372, width=104, height=24)
movement_menu.png found: Box(left=401, top=573, width=142, height=20)
rotation_menu.png found: Box(left=628, top=573, width=125, height=20)
(2206, 11890, -14731) (0, -38, -31)
objects_menu.png found: Box(left=315, top=372, width=144, height=24)
tools_menu.png found: Box(left=557, top=372, width=104, height=24)
movement_menu.png found: Box(left=401, top=573, width=142, height=20)
rotation_menu.png found: Box(left=628, top=573, width=125, height=20)
(2001, 11890, -15013) (0, -34, -3

In [ ]:
image = pyautogui.screenshot()
image.save('testing.png')

In [ ]:
ep = straight_panal_extrapolator((0, 0, 0), (0, 0, 0), 4)

In [ ]:
for p, r in ep():
    print(p, r)

(400, 0, 0) (0, 0, 0)
(800, 0, 0) (0, 0, 0)
(1200, 0, 0) (0, 0, 0)


In [ ]:
curv_extrapolator((0, 5000, 0), (0, -90, -31), 1000, 5)

0 5000 -1000


In [ ]:
ce = curv_extrapolator((-1546, 12890, 1235), (0, 0, -31), 1000, 5)

In [ ]:
print((-1546, 12890, 1235), (0, 0, -31))
for loc, rot in ce():
    print(loc, rot)

(-1546, 12890, 1235) (0, 0, -31)
(-1542, 12890, 1148) (0, 5.0, -31)
(-1531, 12890, 1061) (0, 10.0, -31)
(-1512, 12890, 976) (0, 15.0, -31)
(-1486, 12890, 893) (0, 20.0, -31)


In [ ]:
math.degrees(math.atan(400.0 / float(2000)))

11.309932474020215

In [ ]:
800 * math.cos(math.radians(31))

685.7338405616898

In [ ]:
math.cos(math.radians(31))

0.8571673007021123

In [74]:
mod_rot(-66666)

-66

In [85]:
fce = flex_curv_extrapolator((-1546, 12890, 1235), (0, 0, -31), [(1000, 2), (500, 3), (1000, 2)])

In [87]:
print((-1546, 12890, 1235), (0, 0, -31))
for loc, rot in fce():
    print(loc, rot)

(-1546, 12890, 1235) (0, 0, -31)
(-1516, 12890, 993) (0, 14, -31)
(-1277, 12890, 553) (0, 43, -31)
(-1130, 12890, 445) (0, 64, -31)
(-782, 12890, 411) (0, 105, -31)
(-424, 12890, 782) (0, 167, -31)
(-398, 12890, 1024) (0, -179, -31)
(-532, 12890, 1507) (0, -150, -31)
